In [1]:
from config import spotify_client_ID, spotify_client_secret, my_spotify_uri, weather_api, my_redirect_url, username, scope
from spotipy.oauth2 import SpotifyClientCredentials
from json.decoder import JSONDecodeError

import datetime 
import openweathermapy.core as owm
import requests
import spotipy
import pprint
import sys
import re
import os
import pandas as pd
import spotipy.util as util
import json
import billboard
import time

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
playlist_df = pd.read_csv('Top 50 URI.csv')
playlist_df = playlist_df.set_index('Country')

playlist_ID_list = list(playlist_df['URI'])

playlist_df['URI'][0]

'spotify:user:spotifycharts:playlist:37i9dQZEVXbMDoHDwVN2tF'

In [4]:
try:
    util.prompt_for_user_token(username, scope, client_id=spotify_client_ID,client_secret=spotify_client_secret,redirect_uri=my_redirect_url)
except (AttributeError, JSONDecodeError):
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username, scope, client_id=spotify_client_ID,client_secret=spotify_client_secret,redirect_uri=my_redirect_url)

In [5]:
token = util.oauth2.SpotifyClientCredentials(client_id= spotify_client_ID, client_secret=spotify_client_secret )

cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)

In [6]:
# token = util.oauth2.SpotifyClientCredentials(client_id= '7efe5b269e454abe887ee86a0c81601e'
# , client_secret='908f8904f0b749f9b49ceacc6f89fd3b')

In [7]:
# a = util.prompt_for_user_token(username,scope = 'user-read-playback-state', client_id=spotify_client_ID,client_secret=spotify_client_secret,redirect_uri=my_redirect_url)

In [9]:
# Generates list containing names of ALL PLAYLISTS in charts 
# Get TOP SONG from each playlist

playlist_list = []
top_songs = []
top_artists = []
song_uri_list = []

for ID in playlist_ID_list:
    playlist = spotify.user_playlist('spotifycharts', playlist_id = ID)
    name = playlist['name']
    playlist_list.append(name)
    
    top_song = playlist['tracks']['items'][0]['track']['name']
    top_songs.append(top_song)
    
    top_artist = playlist['tracks']['items'][0]['track']['artists'][0]['name']
    top_artists.append(top_artist)
    
    song_uri = playlist['tracks']['items'][0]['track']['uri']
    song_uri_list.append(song_uri)
    
    
   

In [10]:
danceability = []
energy = []
acousticness = []
valence = []
tempo = []
instru = []

for uri in song_uri_list:
    features = spotify.audio_features(uri)[0]
    danceability.append(features['danceability'])
    energy.append(features['energy'])
    valence.append(features['valence'])
    acousticness.append(features['acousticness'])
    tempo.append(features['tempo'])
    instru.append(features['instrumentalness'])

SSLError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/audio-features/?ids=2XW4DbS6NddZxRPm5rMCeY (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [ ]:
playlist_df['Top Song']= top_songs
playlist_df['Top Artist'] = top_artists
playlist_df['Song URI'] = song_uri_list
playlist_df['Danceability'] = danceability
playlist_df['Energy'] = energy
playlist_df['Acousticness'] = acousticness
playlist_df['Valence'] = valence
playlist_df['Tempo'] = tempo
playlist_df['Intrumentalness'] = instru

playlist_df

In [ ]:
top_songs_artists = zip(top_songs, top_artists)
lyrics = []

for song, artist in top_songs_artists:
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    words = requests.get(url).json()
    lyrics.append(words)
    time.sleep(1)
    
    



In [ ]:

for i in range(len(lyrics)):
    try:
        lyrics[i] = lyrics[i]['lyrics']
        lyrics[i] = lyrics[i].replace('\n', ' ')
    except:
        lyrics[i] = 'N/A'



In [ ]:
playlist_df['Lyrics'] = lyrics

In [ ]:
sentiments = []

for words in lyrics:
    
    results = analyzer.polarity_scores(words)
    compound = results["compound"]
    sentiments.append(compound)
    time.sleep(1)
print(sentiments)

In [ ]:
playlist_df['Compound Sentiment'] = sentiments
playlist_df